In [439]:
import os
import string
import stopwords as stopwords
import snscrape.modules.twitter as sntwitter
import pandas as pd
from tqdm import tqdm
import time
import re
import preprocessor as p
from nltk.tokenize import word_tokenize
from textblob import TextBlob
from nltk.corpus import stopwords
import pygsheets
from tkinter import *
from tkinter.ttk import *
import pycountry
from tkinter import messagebox
from tkinter import ttk
import webbrowser
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.utils.np_utils import to_categorical
from sklearn.utils import resample
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix,classification_report


In [440]:
import pandas as pd
df = pd.read_excel("C:/Users/lenovo/Desktop/Project - Major/Data_for_train.xlsx")

In [441]:
df = df[['text_sentiment_label', 'Tweets_description']]
df

,text_sentiment_label,Tweets_description
0,negative,"@SouthwestAir I am scheduled for the morning, ..."
1,positive,@SouthwestAir seeing your workers time in and ...
2,positive,@united Flew ORD to Miami and back and had gr...
3,negative,@SouthwestAir @dultch97 that's horse radish ðŸ...
4,negative,@united so our flight into ORD was delayed bec...
...,...,...
260270,neutral,why these 456 crores paid neerav modi not reco...
260271,neutral,dear rss terrorist payal gawar what about modi...
260272,negative,did you cover her interaction forum where she ...
260273,negative,there big project came into india modi dream p...


In [442]:
#HappyEmoticons
emoticons_happy = set([
    ':-)', ':)', ';)', ':o)', ':]', ':3', ':c)', ':>', '=]', '8)', '=)', ':}',
   ':^)', ':-D', ':D', '8-D', '8D', 'x-D', 'xD', 'X-D', 'XD', '=-D', '=D',
    '=-3', '=3', ':-))', ":'-)", ":')", ':*', ':^*', '>:P', ':-P', ':P', 'X-P',
    'x-p', 'xp', 'XP', ':-p', ':p', '=p', ':-b', ':b', '>:)', '>;)', '>:-)',
    '<3'
    ])

# Sad Emoticons
emoticons_sad = set([
    ':L', ':-/', '>:/', ':S', '>:[', ':@', ':-(', ':[', ':-||', '=L', ':<',
    ':-[', ':-<', '=\\', '=/', '>:(', ':(', '>.<', ":'-(", ":'(", ':\\', ':-c',
    ':c', ':{', '>:\\', ';('
    ])

#Emoji patterns
emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)

#combine sad and happy emoticons
emoticons = emoticons_happy.union(emoticons_sad)
stop_words = set(stopwords.words('english'))

In [443]:
import nltk
import re
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
nltk.download('wordnet')
nltk.download('stopwords')

lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def remove_mentions(text):
    # remove mentions (words that start with '@')
    return re.sub(r'@\w+', '', text)


def clean_tweets(tweet):
    # Remove non-ASCII characters
    tweet = remove_mentions(tweet)
    tweet = re.sub('[^\x00-\x7F]+', ' ', tweet)
    
    # Remove URLs
    tweet = re.sub(r'https?:\/\/\S+', '', tweet)
    tweet = emoji_pattern.sub(r'', tweet)
    tweet = tweet.translate(str.maketrans('', '', string.punctuation))
    
    # Tokenize tweet
    word_tokens = word_tokenize(tweet.lower())
    
    # Lemmatize words and remove stop words, emoticons, and punctuation
    filtered_tweet = []
    for w in word_tokens:
        if w not in stop_words and w not in string.punctuation:
            w = re.sub(r'@\w+', '', w)
            filtered_tweet.append(w)

    return ' '.join(filtered_tweet)

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [444]:
feature = pd.read_excel("C:/Users/lenovo/Desktop/Streamlit tutorial/Text_Sentiment_analysis/features.xlsx")

In [445]:
def process_feature(text):
    tokens = nltk.word_tokenize(text)
    lowercase_tokens = [token.lower() for token in tokens]
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in lowercase_tokens]
    unique_tokens = list(set(lemmatized_tokens))
    processed_text = " ".join(unique_tokens)
    return processed_text

In [446]:
feature['Feeling'] = feature['Feeling'].apply(str)
feature['Feeling'] = feature['Feeling'].apply(process_feature)

In [447]:
feature_list = feature['Feeling'].tolist()
feature['Feeling']
feature_list

['absorbed',
 'beguiled',
 'busy',
 'charmed',
 'cheered-up',
 'convulsed',
 'delighted',
 'diverted',
 'engaged',
 'entertained',
 'glad',
 'interested',
 'involved',
 'pleased',
 'preoccupied',
 'regaled',
 'wowed',
 'beaming',
 'blessed',
 'blissful',
 'carefree',
 'cheerful',
 'confident',
 'content',
 'euphoric',
 'exhilarated',
 'glowing',
 'gratified',
 'inclined',
 'joy',
 'powerful',
 'radiant',
 'secure',
 'self-accepting',
 'strong',
 'thrilled',
 'amicable',
 'anxiety-free',
 'calm',
 'committed',
 'composed',
 'cordial',
 'flexible',
 'harmonious',
 'isolated',
 'patient',
 'pleasant',
 'productive',
 'quiet',
 'relaxed',
 'restful',
 'serene',
 'soothed',
 'stable',
 'tranquil',
 'undisturbed',
 'untroubled',
 'unworried',
 'acceptable',
 'amused',
 'contented',
 'elated',
 'good',
 'grateful',
 'happy',
 'lovely',
 'nice',
 'overjoyed',
 'satisfied',
 'self-approving',
 'superior',
 'thankful',
 'well-pleased',
 'athletic',
 'buff',
 'compelling',
 'controlling',
 'domin

In [448]:
from nltk import pos_tag
from nltk.corpus import wordnet
from nltk import pos_tag

lemmatizer = WordNetLemmatizer()

def get_simple_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN 

# create a dictionary to store the mappings between the original words and their cleaned versions
word_dict = {}

def get_word(word):
    if word not in word_dict:
        pos = pos_tag([word])[0][1]
        if pos.startswith('J'):
            clean_word = lemmatizer(word, get_simple_pos(pos))[0]
            word_dict[word] = clean_word.lower()
        else:
            word_dict[word] = ""
    return word_dict[word]

def get_tag(text):
    output_words = [get_word(word) for word in text]
    return output_words


In [449]:
# def feature_extract(text):
#     tokens = nltk.word_tokenize(text)
#     pos_tags = pos_tag(tokens)  # perform POS tagging on the tokens
#     extracted_features = []
#     extracted_text.append([tag[1] for tag in pos_tags])
#     return extracted_text


In [450]:
df['Tweets_description'] = df['Tweets_description'].apply(str)

In [451]:
df['Tweets_description'] = df['Tweets_description'].apply(clean_tweets)
df

,text_sentiment_label,Tweets_description
0,negative,scheduled morning 2 days fact yesnot sure even...
1,positive,seeing workers time time going beyond love fly...
2,positive,flew ord miami back great crew service legs th...
3,negative,thats horse radish
4,negative,flight ord delayed air force one last flight s...
...,...,...
260270,neutral,456 crores paid neerav modi recovered congress...
260271,neutral,dear rss terrorist payal gawar modi killing 10...
260272,negative,cover interaction forum left
260273,negative,big project came india modi dream project happ...


In [452]:
print(df[df['text_sentiment_label'] == 'positive'].size)
print(df[df['text_sentiment_label'] == 'negative'].size)
print(df[df['text_sentiment_label'] == 'neutral'].size)

200842
170946
148762


In [453]:
max_fatures = 3000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(df['Tweets_description'].values)
X = tokenizer.texts_to_sequences(df['Tweets_description'].values)
X = pad_sequences(X)
X[:2]
input_len = X.shape[1]
input_len

162

In [ ]:
embed_dim = 128
lstm_out = 196
num_classes = 3
model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = input_len))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(3, activation='sigmoid')) # Change activation function to sigmoid
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())


In [454]:
embed_dim = 128
lstm_out = 196
num_classes = 3
model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = input_len))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(3,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_16 (Embedding)    (None, 162, 128)          384000    
                                                                 
 spatial_dropout1d_3 (Spatia  (None, 162, 128)         0         
 lDropout1D)                                                     
                                                                 
 lstm_6 (LSTM)               (None, 196)               254800    
                                                                 
 dense_15 (Dense)            (None, 3)                 591       
                                                                 
Total params: 639,391
Trainable params: 639,391
Non-trainable params: 0
_________________________________________________________________
None


In [455]:
from keras.utils import to_categorical
label_dict = {'positive': 0, 'negative': 1, 'neutral': 2}
df['label_int'] = df['text_sentiment_label'].replace(label_dict)
Y = to_categorical(df['label_int'], num_classes=num_classes)
# Y = pd.get_dummies(df['text_sentiment_label']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.20, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(208220, 162) (208220, 3)
(52055, 162) (52055, 3)


In [456]:
batch_size = 162
model.fit(X_train, Y_train, epochs = 15, batch_size=batch_size, verbose = 1)

Epoch 1/15
1286/1286 [==============================] - 1322s 1s/step - loss: 0.7011 - accuracy: 0.6991
Epoch 2/15
1286/1286 [==============================] - 1420s 1s/step - loss: 0.5650 - accuracy: 0.7729
Epoch 3/15
1286/1286 [==============================] - 2755s 2s/step - loss: 0.5321 - accuracy: 0.7897
Epoch 4/15
1286/1286 [==============================] - 2536s 2s/step - loss: 0.5081 - accuracy: 0.8008
Epoch 5/15
1286/1286 [==============================] - 2051s 2s/step - loss: 0.4915 - accuracy: 0.8089
Epoch 6/15
1286/1286 [==============================] - 2029s 2s/step - loss: 0.4745 - accuracy: 0.8161
Epoch 7/15
1286/1286 [==============================] - 2014s 2s/step - loss: 0.4591 - accuracy: 0.8222
Epoch 8/15
1286/1286 [==============================] - 2097s 2s/step - loss: 0.4455 - accuracy: 0.8284
Epoch 9/15
1286/1286 [==============================] - 2053s 2s/step - loss: 0.4322 - accuracy: 0.8325
Epoch 10/15
1286/1286 [==============================] - 1971s 2

In [457]:
# Y_pred = model.predict_classes(X_test,batch_size = batch_size)
Y_pred = np.argmax(model.predict(X_test),axis=1)
df_test = pd.DataFrame({'true': Y_test.tolist(), 'pred':Y_pred})
df_test['true'] = df_test['true'].apply(lambda x: np.argmax(x))
print("confusion matrix",confusion_matrix(df_test.true, df_test.pred))
print(classification_report(df_test.true, df_test.pred))

1627/1627 [==============================] - 193s 118ms/step
confusion matrix [[16389  1660  2142]
 [  591 15015  1376]
 [ 1480  1777 11625]]
              precision    recall  f1-score   support

           0       0.89      0.81      0.85     20191
           1       0.81      0.88      0.85     16982
           2       0.77      0.78      0.77     14882

    accuracy                           0.83     52055
   macro avg       0.82      0.83      0.82     52055
weighted avg       0.83      0.83      0.83     52055



In [460]:
Y_pred = np.argmax(model.predict(X_test))
Y_pred

1627/1627 [==============================] - 530s 325ms/step


137027

In [458]:
model.save('sentiment_analysis_trail_model.h5')

In [461]:
# Separate majority and minority classes
data_majority = df[df['text_sentiment_label'] == 'neutral' or data['sentiment'] == 'positive']
data_minority = df[df['text_sentiment_label'] == 'negative']

bias = data_minority.shape[0]/data_majority.shape[0]
# lets split train/test data first then 
train = pd.concat([data_majority.sample(frac=0.8,random_state=200),
         data_minority.sample(frac=0.8,random_state=200)])
test = pd.concat([data_majority.drop(data_majority.sample(frac=0.8,random_state=200).index),
        data_minority.drop(data_minority.sample(frac=0.8,random_state=200).index)])

train = shuffle(train)
test = shuffle(test)

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [ ]:
print('positive data in training:',(train.sentiment == 'positive' or train.sentiment == 'neutral').sum())
print('negative data in training:',(train.sentiment == 'negative').sum())
print('positive data in test:',(test.sentiment == 'positive' or train.sentiment == 'neutral').sum())
print('negative data in test:',(test.sentiment == 'negative').sum())

In [ ]:
#Separate majority and minority classes in training data for upsampling 
data_majority = train[train['sentiment'] == 'Negative']
data_minority = train[train['sentiment'] == 'Positive']

print("majority class before upsample:",data_majority.shape)
print("minority class before upsample:",data_minority.shape)

# Upsample minority class
data_minority_upsampled = resample(data_minority, 
                                 replace=True,     # sample with replacement
                                 n_samples= data_majority.shape[0],    # to match majority class
                                 random_state=123) # reproducible results
 
# Combine majority class with upsampled minority class
data_upsampled = pd.concat([data_majority, data_minority_upsampled])
 
# Display new class counts
print("After upsampling\n",data_upsampled.sentiment.value_counts(),sep = "")

max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(df['Tweets_description'].values) # training with whole data

X_train = tokenizer.texts_to_sequences(data_upsampled['Tweets_description'].values)
X_train = pad_sequences(X_train,maxlen=29)
Y_train = pd.get_dummies(data_upsampled['text_sentiment_label']).values
print('x_train shape:',X_train.shape)

X_test = tokenizer.texts_to_sequences(test['Tweets_description'].values)
X_test = pad_sequences(X_test,maxlen=29)
Y_test = pd.get_dummies(test['sentiment']).values
print("x_test shape", X_test.shape)

In [ ]:
# model
embed_dim = 128
lstm_out = 192

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X_train.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.4, recurrent_dropout=0.4))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

In [ ]:
batch_size = 128
# also adding weights
class_weights = {0: 1 ,
                1: 1.6/bias }
model.fit(X_train, Y_train, epochs = 15, batch_size=batch_size, verbose = 1,
          class_weight=class_weights)

In [ ]:
Y_pred = model.predict_classes(X_test,batch_size = batch_size)
df_test = pd.DataFrame({'true': Y_test.tolist(), 'pred':Y_pred})
df_test['true'] = df_test['true'].apply(lambda x: np.argmax(x))
print("confusion matrix",confusion_matrix(df_test.true, df_test.pred))
print(classification_report(df_test.true, df_test.pred))

In [ ]:
# running model to few more epochs
model.fit(X_train, Y_train, epochs = 15, batch_size=batch_size, verbose = 1,
          class_weight=class_weights)
Y_pred = model.predict_classes(X_test,batch_size = batch_size)
df_test = pd.DataFrame({'true': Y_test.tolist(), 'pred':Y_pred})
df_test['true'] = df_test['true'].apply(lambda x: np.argmax(x))
print("confusion matrix",confusion_matrix(df_test.true, df_test.pred))
print(classification_report(df_test.true, df_test.pred))

In [ ]:
twt = ['keep up the good work']
#vectorizing the tweet by the pre-fitted tokenizer instance
twt = tokenizer.texts_to_sequences(twt)
#padding the tweet to have exactly the same shape as `embedding_2` input
twt = pad_sequences(twt, maxlen=29, dtype='int32', value=0)
print(twt)
sentiment = model.predict(twt,batch_size=1,verbose = 2)[0]
if(np.argmax(sentiment) == 0):
    print("negative")
elif (np.argmax(sentiment) == 1):
    print("positive")

In [407]:
# running model to few more epochs
from sklearn.utils import class_weight
model.fit(X_train, Y_train, epochs = 15, batch_size=batch_size, verbose = 1,
          class_weight=class_weights)
Y_pred = model.predict_classes(X_test,batch_size = batch_size)
df_test = pd.DataFrame({'true': Y_test.tolist(), 'pred':Y_pred})
df_test['true'] = df_test['true'].apply(lambda x: np.argmax(x))
print("confusion matrix",confusion_matrix(df_test.true, df_test.pred))
print(classification_report(df_test.true, df_test.pred))

NameError: name 'class_weights' is not defined

In [323]:
train_pos = []
test_pos = []
for text in train_data:
    tokens = word_tokenize(text)
    pos_tags = nltk.pos_tag(tokens)
    train_pos.append([tag[1] for tag in pos_tags])

for text in test_data:
    tokens = word_tokenize(text)
    pos_tags = nltk.pos_tag(tokens)
    test_pos.append([tag[1] for tag in pos_tags])

pos_dict = {'CC': 1, 'CD': 2, 'DT': 3, 'EX': 4, 'FW': 5, 'IN': 6, 'JJ': 7, 'JJR': 8, 'JJS': 9, 'LS': 10,
            'MD': 11, 'NN': 12, 'NNS': 13, 'NNP': 14, 'NNPS': 15, 'PDT': 16, 'POS': 17, 'PRP': 18, 'PRP$': 19,
            'RB': 20, 'RBR': 21, 'RBS': 22, 'RP': 23, 'SYM': 24, 'TO': 25, 'UH': 26, 'VB': 27, 'VBD': 28,
            'VBG': 29, 'VBN': 30, 'VBP': 31, 'VBZ': 32, 'WDT': 33, 'WP': 34, 'WP$': 35, 'WRB': 36, '$' : 37}

# Convert the POS tags to their corresponding numeric values
train_pos_num = [[pos_dict[tag] for tag in pos_tags] for pos_tags in train_pos]
test_pos_num = [[pos_dict[tag] for tag in pos_tags] for pos_tags in test_pos]


In [324]:
# # Feature extraction
# from sklearn.feature_extraction.text import TfidfVectorizer

# X = vectorizer.fit_transform(df['Text'])
# X_array = X.toarray()
# df['Text'] = list(X_array)
# df['Text']

In [325]:
# # df['X_train'] = df['Text']
# label_dict = {'negative': 0, 'neutral': 1, 'positive': 2}
# Y_label = df['text_sentiment_label'].tolist()
# Y = [label_dict[label] for label in Y_label]

In [334]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras import models
from tensorflow.keras import layers

from tensorflow.keras.preprocessing.sequence import pad_sequences

train_pos_num = np.array(train_pos_num)

# Pad the sequences to the same length
max_length = max([len(seq) for seq in train_pos_num])
max_length_1 = max([len(seq) for seq in test_pos_num])
max_length = max(max_length, max_length_1)
train_pos_pad = pad_sequences(train_pos_num, maxlen=max_length, padding='post')
test_pos_pad = pad_sequences(test_pos_num, maxlen=max_length, padding='post')

# Convert your labels to numeric values
label_dict = {'negative': 0, 'neutral': 1, 'positive': 2}
train_labels_num = [label_dict[label] for label in train_labels]
test_labels_num = [label_dict[label] for label in test_labels]
train_labels_num = np.array(train_labels_num)
# Define your LSTM model
model = models.Sequential([
    Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=16, input_length=max_length),
    Flatten(),
    layers.Dense(10, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])
# Train the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(train_pos_pad, train_labels_num, epochs=10, batch_size=32, validation_split=0.2)

# Save the model
model.save('sentiment_analysis1_model.h5')


Epoch 1/10


InvalidArgumentError: Graph execution error:

Detected at node 'sequential_10/embedding_11/embedding_lookup' defined at (most recent call last):
    File "C:\Users\lenovo\anaconda3\envs\tensorflow_env\lib\runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\lenovo\anaconda3\envs\tensorflow_env\lib\runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "C:\Users\lenovo\anaconda3\envs\tensorflow_env\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "C:\Users\lenovo\anaconda3\envs\tensorflow_env\lib\site-packages\traitlets\config\application.py", line 992, in launch_instance
      app.start()
    File "C:\Users\lenovo\anaconda3\envs\tensorflow_env\lib\site-packages\ipykernel\kernelapp.py", line 711, in start
      self.io_loop.start()
    File "C:\Users\lenovo\anaconda3\envs\tensorflow_env\lib\site-packages\tornado\platform\asyncio.py", line 215, in start
     

In [ ]:
# import numpy as np

# # Define a function to convert a sentence to a feature vector
# def sentence_to_vector(sentence, model):
#     words = sentence.split()
#     vectors = [model[word] for word in words if word in model]
#     if len(vectors) == 0:
#         return np.zeros(model.vector_size)
#     else:
#         return np.mean(vectors, axis=0)

# # Example usage
# sentence = "This is a sample sentence for feature extraction"
# vector = sentence_to_vector(sentence, model)
# print(vector)


In [ ]:
# from gensim.models import Word2Vec
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Embedding, Flatten, Dense
# MAX_SEQUENCE_LENGTH = 100

# # Train the Word2Vec model
# sentences = [["this", "is", "the", "first", "sentence", "for", "word2vec"],
#              ["this", "is", "the", "second", "sentence"],
#              ["yet", "another", "sentence"],
#              ["one", "more", "sentence"],
#              ["and", "the", "final", "sentence"]]
# model = Word2Vec(sentences, min_count=1)

# # Define the Keras model
# vocab_size = len(model.wv.key_to_index)

# embedding_size = model.vector_size
# model_keras = Sequential([
#     Embedding(input_dim=vocab_size, output_dim=embedding_size, input_length=MAX_SEQUENCE_LENGTH,
#               weights=[model.wv.vectors], trainable=False),
#     Flatten(),
#     Dense(1, activation='sigmoid')
# ])

# # Compile and train the Keras model
# model_keras.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# model_keras.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=32)


In [ ]:
# from sklearn.svm import SVC
# svcv = SVC(C=0.2,kernel='linear')

# X_train, X_test, Y_train, Y_test = train_test_split(df['X_train'], df['Y_train'], test_size=0.2, random_state=42)
# # Convert the text to sequences of integers
# X_train_seq = tokenizer.texts_to_sequences(X_train)
# X_test_seq = tokenizer.texts_to_sequences(X_test)

# svcv.fit(X_train_seq, Y_train)
# svcv_scr=svcv.score(X_test_seq, Y_test)
# svcv_scr